In [2]:
pip install xgboost

     --------------------------------------- 70.9/70.9 MB 15.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import time

In [2]:
tiempo = pd.read_parquet('dataframe_final_1970-2023_alicante_alicante.parquet')

In [3]:
# 70% del tiempo para el train, 30% validacion (hasta 31 diciembre 2022)
# el conjunto de test cogeremos 2023

fecha_test = '2007-12-31'

In [4]:
# las variables del día siguiente las eliminamos y también las correspondientes a la localización ya que estamos solo en Alicante

model_columns = list(set(tiempo.columns) - set(['tmed_mañana', 'prec_mañana', 'tmin_mañana', 'horatmin_mañana', 'tmax_mañana',
                                                                                    'horatmax_mañana', 'dir_mañana', 'velmedia_mañana', 'racha_mañana', 'horaracha_mañana', 
                                                                                    'sol_mañana', 'presMax_mañana', 'horaPresMax_mañana', 'presMin_mañana', 'horaPresMin_mañana',
                                                                                    'indicativo', 'nombre', 'provincia', 'altitud', 'fecha']))

In [5]:
tiempo.horatmin = tiempo.horatmin.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax = tiempo.horatmax.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha = tiempo.horaracha.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaPresMax = tiempo.horaPresMax.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin = tiempo.horaPresMin.apply(lambda x: x.hour if x is not None else np.nan)

tiempo.horatmin_1 = tiempo.horatmin_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_2 = tiempo.horatmin_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_3 = tiempo.horatmin_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_mañana = tiempo.horatmin_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horatmax_1 = tiempo.horatmax_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_2 = tiempo.horatmax_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_3 = tiempo.horatmax_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_mañana = tiempo.horatmax_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horaracha_1 = tiempo.horaracha_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_2 = tiempo.horaracha_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_3 = tiempo.horaracha_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_mañana = tiempo.horaracha_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horaPresMax_1 = tiempo.horaPresMax_1.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_2 = tiempo.horaPresMax_2.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_3 = tiempo.horaPresMax_3.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_mañana = tiempo.horaPresMax_mañana.apply(lambda x: x.hour if x is not None else np.nan)

tiempo.horaPresMin_1 = tiempo.horaPresMin_1.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_2 = tiempo.horaPresMin_2.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_3 = tiempo.horaPresMin_3.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_mañana = tiempo.horaPresMin_mañana.apply(lambda x: x.hour if x is not None else np.nan)

In [6]:
tiempo['año'] = tiempo['fecha'].dt.year
tiempo['mes'] = tiempo['fecha'].dt.month
tiempo['dia'] = tiempo['fecha'].dt.day

In [7]:
train = tiempo[tiempo.fecha <= fecha_test].fillna(-1)
test = tiempo[tiempo.fecha > fecha_test].fillna(-1)

Dejamos los parámetros de regularización por defecto

Hay que pasar los datos a float o int

In [8]:
for columna in model_columns:
    train[columna] = train[columna].astype(float)
    test[columna] = test[columna].astype(float)

## Target: tmax_mañana

In [9]:
metrics = {}
for n_estimators in [5, 10, 20, 50, 100, 200, 500]:
    for max_depth in [5, 10, 20, 40]:
        for eta in [0.01 ,0.05, 0.1]:
            start_time = time.time()
            model = XGBRegressor(max_depth = max_depth, eta = eta, n_estimators = n_estimators )
            model.fit(train[model_columns],train.tmax_mañana);

            train_pred = model.predict(train[model_columns])
            test_pred = model.predict(test[model_columns])

            metrics['XGB_'+ str(n_estimators)+'_'+str(max_depth)+'_'+str(eta)] = {
                'Train_MAE': mean_absolute_error(train.tmax_mañana, train_pred),
                'Test_MAE': mean_absolute_error(test.tmax_mañana, test_pred),
                'Run_Time': time.time() - start_time
            }

metrics_XGB = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_XGB['delta'] = metrics_XGB.Test_MAE - metrics_XGB.Train_MAE
metrics_XGB

,Run_Time,Train_MAE,Test_MAE,delta
XGB_5_5_0.01,0.734615,21.610485,22.156490,0.546005
XGB_5_5_0.05,0.339577,17.588646,18.049306,0.460660
XGB_5_5_0.1,0.256789,13.428519,13.799850,0.371330
XGB_5_10_0.01,0.217887,21.610537,22.156616,0.546079
XGB_5_10_0.05,0.317125,17.589018,18.048978,0.459961
...,...,...,...,...
XGB_500_20_0.05,58.871270,0.000448,1.677743,1.677295
XGB_500_20_0.1,29.278471,0.000429,1.709133,1.708704
XGB_500_40_0.01,91.530411,0.219626,1.707116,1.487490
XGB_500_40_0.05,70.831563,0.000263,1.688366,1.688103


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos °C de media se equivoca más en el Test.

In [10]:
metrics_XGB.to_parquet('xgb_tmax.parquet')

## Target: tmin_mañana

In [12]:
metrics = {}
for n_estimators in [5, 10, 20, 50, 100, 200, 500]:
    for max_depth in [5, 10, 20, 40]:
        for eta in [0.01 ,0.05, 0.1]:
            start_time = time.time()
            model = XGBRegressor(max_depth = max_depth, eta = eta, n_estimators = n_estimators )
            model.fit(train[model_columns],train.tmin_mañana);

            train_pred = model.predict(train[model_columns])
            test_pred = model.predict(test[model_columns])

            metrics['XGB_'+ str(n_estimators)+'_'+str(max_depth)+'_'+str(eta)] = {
                'Train_MAE': mean_absolute_error(train.tmin_mañana, train_pred),
                'Test_MAE': mean_absolute_error(test.tmin_mañana, test_pred),
                'Run_Time': time.time() - start_time
            }

metrics_XGB = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_XGB['delta'] = metrics_XGB.Test_MAE - metrics_XGB.Train_MAE
metrics_XGB

,Run_Time,Train_MAE,Test_MAE,delta
XGB_5_5_0.01,0.185992,11.702292,12.653278,0.950987
XGB_5_5_0.05,0.209416,9.531493,10.357592,0.826098
XGB_5_5_0.1,0.197451,7.292328,7.985280,0.692951
XGB_5_10_0.01,0.399255,11.703075,12.655196,0.952121
XGB_5_10_0.05,0.473037,9.534024,10.369050,0.835026
...,...,...,...,...
XGB_500_20_0.05,36.262076,0.000360,1.366246,1.365887
XGB_500_20_0.1,16.736659,0.000361,1.382317,1.381957
XGB_500_40_0.01,55.197271,0.125579,1.417316,1.291737
XGB_500_40_0.05,41.693222,0.000259,1.372276,1.372017


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos °C de media se equivoca más en el Test.

In [13]:
metrics_XGB.to_parquet('xgb_tmin.parquet')

## Target: prec_mañana

In [14]:
metrics = {}
for n_estimators in [5, 10, 20, 50, 100, 200, 500]:
    for max_depth in [5, 10, 20, 40]:
        for eta in [0.01 ,0.05, 0.1]:
            start_time = time.time()
            model = XGBRegressor(max_depth = max_depth, eta = eta, n_estimators = n_estimators )
            model.fit(train[model_columns],train.prec_mañana);

            train_pred = model.predict(train[model_columns])
            test_pred = model.predict(test[model_columns])

            metrics['XGB_'+ str(n_estimators)+'_'+str(max_depth)+'_'+str(eta)] = {
                'Train_MAE': mean_absolute_error(train.prec_mañana, train_pred),
                'Test_MAE': mean_absolute_error(test.prec_mañana, test_pred),
                'Run_Time': time.time() - start_time
            }

metrics_XGB = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_XGB['delta'] = metrics_XGB.Test_MAE - metrics_XGB.Train_MAE
metrics_XGB

,Run_Time,Train_MAE,Test_MAE,delta
XGB_5_5_0.01,0.141611,1.256029,1.198043,-0.057986
XGB_5_5_0.05,0.117675,1.263370,1.217814,-0.045556
XGB_5_5_0.1,0.123660,1.272611,1.240420,-0.032191
XGB_5_10_0.01,0.192471,1.244214,1.201501,-0.042713
XGB_5_10_0.05,0.199752,1.205845,1.228016,0.022171
...,...,...,...,...
XGB_500_20_0.05,39.476459,0.000395,1.715610,1.715214
XGB_500_20_0.1,17.525392,0.000391,1.719102,1.718711
XGB_500_40_0.01,85.426167,0.043763,1.710239,1.666476
XGB_500_40_0.05,54.142236,0.000230,1.766592,1.766362


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos mm de media se equivoca más en el Test.

In [15]:
metrics_XGB.to_parquet('xgb_prec.parquet')

## Target: sol_mañana

In [16]:
metrics = {}
for n_estimators in [5, 10, 20, 50, 100, 200, 500]:
    for max_depth in [5, 10, 20, 40]:
        for eta in [0.01 ,0.05, 0.1]:
            start_time = time.time()
            model = XGBRegressor(max_depth = max_depth, eta = eta, n_estimators = n_estimators )
            model.fit(train[model_columns],train.sol_mañana);

            train_pred = model.predict(train[model_columns])
            test_pred = model.predict(test[model_columns])

            metrics['XGB_'+ str(n_estimators)+'_'+str(max_depth)+'_'+str(eta)] = {
                'Train_MAE': mean_absolute_error(train.sol_mañana, train_pred),
                'Test_MAE': mean_absolute_error(test.sol_mañana, test_pred),
                'Run_Time': time.time() - start_time
            }

metrics_XGB = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_XGB['delta'] = metrics_XGB.Test_MAE - metrics_XGB.Train_MAE
metrics_XGB

,Run_Time,Train_MAE,Test_MAE,delta
XGB_5_5_0.01,0.138619,7.072199,7.802435,0.730236
XGB_5_5_0.05,0.157566,5.919745,6.587982,0.668237
XGB_5_5_0.1,0.119670,4.805961,5.394490,0.588529
XGB_5_10_0.01,0.244484,7.064631,7.807820,0.743190
XGB_5_10_0.05,0.230938,5.867598,6.607808,0.740211
...,...,...,...,...
XGB_500_20_0.05,35.071181,0.000397,2.503882,2.503485
XGB_500_20_0.1,18.430979,0.000363,2.506125,2.505762
XGB_500_40_0.01,81.182312,0.085697,2.563976,2.478279
XGB_500_40_0.05,51.447920,0.000264,2.529501,2.529237


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántas h de media se equivoca más en el Test.

In [17]:
metrics_XGB.to_parquet('xgb_sol.parquet')